In [2]:
import numpy as np
import gym
# TODO: For more gym_bandit environemnts, or use Bsuite Bandits
import gym_bandits

import garage

In [3]:
!pip list | grep garage
!pip list | grep tensorflow

garage                        2020.9.0rc2.dev0    /Users/robvangastel/Git/garage/src
tensorflow                    2.4.1
tensorflow-estimator          2.4.0
tensorflow-probability        0.12.1


## Bandits environments in the OpenAI Gym

In [4]:
banditEnv = gym.make('MultiarmedBandits-v0')
banditEnv

## RL2

In [5]:
from garage import wrap_experiment
from garage.envs import GymEnv #Old object?

# TODO: The current environments, require a meta-RL base class
@wrap_experiment
def rl2_ppo_bandit_meta_test(ctxt, seed=42, meta_batch_size=10, n_epochs=10,
                                  episode_per_task=4):
    return NotImplemented



rl2_ppo_bandit_meta_test()

2021-02-27 18:37:16 | [rl2_ppo_bandit_meta_test] Logging to /Users/robvangastel/Git/meta-learners/notebooks/data/local/experiment/rl2_ppo_bandit_meta_test


NotImplemented

## MAML with RL

In [1]:
import tensorflow as tf

from garage import wrap_experiment
from garage.envs import GymEnv, normalize
from garage.envs.multi_env_wrapper import MultiEnvWrapper
from garage import TFTrainer
from garage.np.baselines import LinearFeatureBaseline
from garage.experiment.deterministic import set_seed
from garage.np.baselines import LinearMultiFeatureBaseline
from garage.sampler import LocalSampler
from garage.tf.algos.te import TaskEmbeddingWorker
from garage.tf.embeddings import GaussianMLPEncoder
from garage.tf.policies import GaussianMLPTaskEmbeddingPolicy
from garage.tf.policies import CategoricalMLPPolicy
from garage.tf.algos import TRPO

@wrap_experiment
def te_ppo_ml1_push(ctxt, seed=42, n_epochs=1, batch_size_per_task=20):
    set_seed(seed)
    envs = [GymEnv('CartPole-v0'), GymEnv('CartPole-v0'), GymEnv('CartPole-v0')]
    env = MultiEnvWrapper(envs)
    
    latent_length = 2
    inference_window = 6
    batch_size = batch_size_per_task
    policy_ent_coeff = 2e-2
    encoder_ent_coeff = 2e-4
    inference_ce_coeff = 5e-2
    max_episode_length = 100
    embedding_init_std = 0.1
    embedding_max_std = 0.2
    embedding_min_std = 1e-6
    policy_init_std = 1.0
    policy_max_std = None
    policy_min_std = None

    with TFTrainer(snapshot_config=ctxt) as trainer:
        policy = CategoricalMLPPolicy(name='policy',
                                      env_spec=env.spec,
                                      hidden_sizes=(32, 32))


        sampler = LocalSampler(agents=policy,
                               envs=env,
                              max_episode_length=200)

        baseline = LinearFeatureBaseline(env_spec=env.spec)
        algo = TRPO(
            env_spec=env.spec,
            sampler=sampler,
            policy=policy,
            baseline=baseline,
            discount=0.99,
            max_kl_step=0.01,
        )
        

        trainer.setup(algo, env)
        trainer.train(n_epochs=n_epochs, batch_size=batch_size, plot=False)


te_ppo_ml1_push()

2021-02-27 21:19:24 | [te_ppo_ml1_push] Logging to /Users/robvangastel/Git/meta-learners/notebooks/data/local/experiment/te_ppo_ml1_push_22


/Users/robvangastel/.local/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
/Users/robvangastel/.conda/envs/meta/lib/python3.7/site-packages/tensorflow/python/keras/legacy_tf_layers/core.py:171: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  warnings.warn('`tf.layers.dense` is deprecated and '
/Users/robvangastel/.conda/envs/meta/lib/python3.7/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1719: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


Instructions for updating:
Prefer Variable.assign which has equivalent behavior in 2.X.
2021-02-27 21:19:25 | [te_ppo_ml1_push] Obtaining samples...
2021-02-27 21:19:26 | [te_ppo_ml1_push] epoch #0 | Optimizing policy...
2021-02-27 21:19:26 | [te_ppo_ml1_push] epoch #0 | Computing loss before
2021-02-27 21:19:26 | [te_ppo_ml1_push] epoch #0 | Computing KL before
2021-02-27 21:19:26 | [te_ppo_ml1_push] epoch #0 | Optimizing
2021-02-27 21:19:26 | [te_ppo_ml1_push] epoch #0 | Start CG optimization: #parameters: 1378, #inputs: 1, #subsample_inputs: 1
2021-02-27 21:19:26 | [te_ppo_ml1_push] epoch #0 | computing loss before
2021-02-27 21:19:26 | [te_ppo_ml1_push] epoch #0 | computing gradient
2021-02-27 21:19:26 | [te_ppo_ml1_push] epoch #0 | gradient computed
2021-02-27 21:19:26 | [te_ppo_ml1_push] epoch #0 | computing descent direction


/Users/robvangastel/.conda/envs/meta/lib/python3.7/site-packages/garage/tf/optimizers/conjugate_gradient_optimizer.py:546: RuntimeWarning: invalid value encountered in float_scalars
  v = rdotr / p.dot(z)


2021-02-27 21:19:26 | [te_ppo_ml1_push] epoch #0 | descent direction computed
2021-02-27 21:19:26 | [te_ppo_ml1_push] epoch #0 | Line search condition violated. Rejecting the step!
2021-02-27 21:19:26 | [te_ppo_ml1_push] epoch #0 | Violated because loss is NaN
2021-02-27 21:19:26 | [te_ppo_ml1_push] epoch #0 | Violated because constraint mean_kl is NaN
2021-02-27 21:19:26 | [te_ppo_ml1_push] epoch #0 | backtrack iters: 14
2021-02-27 21:19:26 | [te_ppo_ml1_push] epoch #0 | optimization finished
2021-02-27 21:19:26 | [te_ppo_ml1_push] epoch #0 | Computing KL after
2021-02-27 21:19:26 | [te_ppo_ml1_push] epoch #0 | Computing loss after
2021-02-27 21:19:26 | [te_ppo_ml1_push] epoch #0 | Fitting baseline...
2021-02-27 21:19:26 | [te_ppo_ml1_push] epoch #0 | Saving snapshot...
2021-02-27 21:19:26 | [te_ppo_ml1_push] epoch #0 | Saved
2021-02-27 21:19:26 | [te_ppo_ml1_push] epoch #0 | Time 1.07 s
2021-02-27 21:19:26 | [te_ppo_ml1_push] epoch #0 | EpochTime 1.07 s
------------------------------